In [1]:
import sys
import csv
import os
import numpy as np
from opentrons import robot, instruments, labware
import itertools
import datetime
from pytz import timezone
import pytz

C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


In [2]:
pacific = timezone('US/Pacific')
print(datetime.datetime.now())
print(datetime.datetime.now(pacific).date())


2020-09-28 16:59:52.519835
2020-09-28


In [21]:
plate_name = 'corning-48-3548'
if plate_name not in labware.list():
    custom_plate = labware.create(
        plate_name,                     # name of you labware
        grid=(8, 6),                    # specify amount of (columns, rows)
        spacing=(13.08, 13.08),               # distances (mm) between each (column, row)
        diameter=11.05,                    # diameter (mm) of each vial on the plate
        depth=17.4,                     # depth (mm) of each vial on the plate
        volume= 1620)                   # volume (uL) of each vial on the plate


# Agilent 201284-100, deep reservoir plate.
plate_name = 'agilent-201284-100'
if plate_name not in labware.list():
    custom_plate = labware.create(
        plate_name,                     # name of your labware
        grid=(6, 1),                    # specify amount of (columns, rows)
        spacing=(18, 18),               # distances (mm) between each (column, row)
        diameter=17.12,                 # diameter (mm) of each vial on the plate
        depth=42.088,                   # depth (mm) of each vial on the plate
        volume=4700)                    # volume (uL) of each vial on the plate
    
plate_name = 'glass-20ml-vial'
if plate_name not in labware.list():
    custom_plate = labware.create(
        plate_name,                     # name of your labware
        grid=(4, 3),                    # specify amount of (columns, rows)
        spacing=(28, 28),               # distances (mm) between each (column, row)
        diameter=16,                    # diameter (mm) of each vial on the plate
        depth=56.5,                     # depth (mm) of each vial on the plate
        volume=18000)                   # volume (uL) of each vial on the plate    



In [41]:
# robot.connect()

robot.home()
robot.reset()
#Original Directory
# og_dir = os.getcwd()

# #Change your_path to your system's ../ot2protocols. You may or may not need r in front. 
# #I couldn't get this to work otherwise. I had to modify ot2protocols.__init__:
#from .ot2protocols import* -> from ot2protocols import*


# your_path = r'C:\Users\lacho\OneDrive - UW\Pozzo-RG-OT2\OT2Protocols\ot2protocol'
# os.chdir(your_path)
# from Components import *
# from A1Stock import *
# from Mineralization_surfactant_oil_sample_class import *


# #Return to original directory
# os.chdir(og_dir


# Loading classes below because I don't want to deal with import functions right now

In [23]:
class Oil_Surfacant_Mineralization:
    
    """
    Class for handling:
    Oil, vol. f
    Solvent, vol. f
    Surfactant, Molarity
    Buffering and/or reducing agent, Molarity (e.g HEPES)
    Precursor Molarity

    Volume - Volume of sample in mL, assuming negligible contribution from stabilizer, buffer, precursor
    
    
    Dictionaries note - all keys should simply be component name. 

    components_dict - dictionary pointing to Component objects with context specific roles
    e.g. {'hydrophobe':Component, 'organic_solvent':Component, 'solvent1':Component, 'stabilizer1':Component}

    volf_dict - dictionary which ties components to their respective volume fractions:
    e.g. {'hydrophobe':float, 'organic_solvent':float...'solvent1':None}

    stock_dict - dictionary for pointing to A1Stock objects. 
    """

    def __init__(self, volume = None, components_dict ={}, volf_dict ={}, molarity_dict = {} ,density = None, stock_dict ={}):
        self.components_dict = components_dict
        self.volf_dict = volf_dict
        self.molarity_dict = molarity_dict
        self.density = density
        self.stock_dict = stock_dict
        self.volume = float(volume)
        self.component_name_dict = {}

        #Initialize dictionaries of calculated properties below. 
        #Note assigned prefix is method for tabulating how much of a component
        #has already been accounted for by stock assignment.
        #Stock transfer is the volume of a stock that needs to be transferred. 
        self.volumes_dict = {}
        self.moles_dict = {} #Obtained by using Molarity (M) and volume of sample (mL)
        self.assigned_volumes_dict = {}
        self.assigned_moles_dict = {}
        self.assigned_stock_transfer_dict = {}
        

        #Iterate and perform calculations below
        for component_key in components_dict:
            self.assigned_volumes_dict[component_key] = 0.0
            self.assigned_moles_dict[component_key] = 0.0
            #Below was necessary because key error occurs when updating assigned volume
            #of a solvent used in a stock. i.e. we need to map 'water' : 'solvent1'
            self.component_name_dict[self.components_dict[component_key].name] = component_key
            # print(self.components_dict[component_key].name, "mapped to ", component_key)
            if component_key in self.volf_dict:
                if component_key in self.molarity_dict:
                    raise Exception("Calculations must be proceed from volf OR molarity")
                self.volumes_dict[component_key]=self.volume*self.volf_dict[component_key]
                # if 'solvent' not in component_key:
                #     self.stock_dict[component_key].wtf_to_volf()
            if component_key in self.molarity_dict:
                self.moles_dict[component_key] = self.molarity_dict[component_key]*self.volume/1000.0
                # self.stock_dict[component_key].wtf_to_molarity()
                # print(component_key, "molarity = , ", self.stock_dict[component_key].componentA_molarity)
        
        for component_key in self.stock_dict:
            if component_key in self.volf_dict and 'solvent' not in component_key:
                #Assign stock transfer volumes for samples which are handled using volume fraction - e.g. emulsified oil.
                self.assigned_stock_transfer_dict[component_key] = self.volumes_dict[component_key]/self.stock_dict[component_key].componentA_volf
                self.assigned_volumes_dict[component_key]+= self.volumes_dict[component_key]
                
                #Now account for the solvent used to transfer component
                assigned_solvent_name = self.stock_dict[component_key].solvent1.name
                assigned_solvent_name = self.component_name_dict[assigned_solvent_name]

                if type(assigned_solvent_name) is not str:
                    raise Exception("Expected assigned solvent name string to use in a dict, but got ", type(assigned_solvent_name))
                self.assigned_volumes_dict[assigned_solvent_name]+= self.volumes_dict[component_key]/self.stock_dict[component_key].solvent1_volf
            
            if component_key in self.molarity_dict:
                #Assign stock transfer volumes for samples which are handled using molarity - e.g. stablizer, precursor, buffer.
                if float(self.moles_dict[component_key]) == 0.0:
                    self.assigned_stock_transfer_dict[component_key] = 0
                else:
                    self.assigned_stock_transfer_dict[component_key] = 1000.0*self.moles_dict[component_key]/self.stock_dict[component_key].componentA_molarity
                self.assigned_moles_dict[component_key] = self.assigned_stock_transfer_dict[component_key]*self.stock_dict[component_key].componentA_molarity/1000.0

                 #Now account for the solvent used to transfer component
                assigned_solvent_name = self.stock_dict[component_key].solvent1.name
                assigned_solvent_name = self.component_name_dict[assigned_solvent_name]
                if type(assigned_solvent_name) != str:
                    raise Exception("Expected assigned solvent name string to use in a dict, but got ", type(assigned_solvent_name))
                self.assigned_volumes_dict[assigned_solvent_name]+= self.assigned_stock_transfer_dict[component_key]            
        for component_key in components_dict:
            if component_key in self.volf_dict:
                #Check for difference between desired and assigned volume. 
                # print(component_key)
                # print(self.volumes_dict)
                # print(self.assigned_volumes_dict)
                # print(self.volumes_dict[component_key] - self.assigned_volumes_dict[component_key])
                if 'solvent' in component_key:
                    missing_solvent_volume = self.volumes_dict[component_key]-self.assigned_volumes_dict[component_key]
                    self.assigned_stock_transfer_dict[component_key] = missing_solvent_volume
                    self.assigned_volumes_dict[component_key] +=missing_solvent_volume
                    # print("component_key", component_key)
                    # print("self.volumes_dict",self.volumes_dict)
                    # print("self.assigned_volumes_dict", self.assigned_volumes_dict)
                    # print("self.assigned_stock_transfer_dict", self.assigned_stock_transfer_dict)
                    # print(self.volumes_dict[component_key] - self.assigned_volumes_dict[component_key])
                    
    def transfer_ml_to_ul(self):
        """
        Convert assigned_transfer_dict from mL to uL
        """
        self.converted_transfer_dict = {}
        for component_key in self.assigned_stock_transfer_dict:
            self.converted_transfer_dict[component_key] = self.assigned_stock_transfer_dict[component_key]*1000.0 
                
class A1Stock:
    '''
    A1_Stock: shorthand for stock with Component A, and 1 solvent.
    class called AB2 would consist of Components A and B, and 2 solvents. 
    Minimum information: 
    
    components_dict - dictionary pointing to Component objects: 
    e.g. {'A':Component, 'solvent1':Component}
    
    wtf_dict - dictionary which ties components to their respective weight fractions:
    e.g. {'A':float, 'solvent1':float}
    
    density - float. By default this is solvent1.density
    
    TODO: 
    -Potential for A1_stock method which estimates density based on volume of component, 
    or A1_stock method which takes interpolates density from dataset of density and wt.f. Component:solvent
    -Function which manages volume/position of A1_stock, i.e. we can assign multiple stock and have the robot 
    switch to new stock position when necessary.
    '''
    
    def __init__(self, components_dict ={}, wtf_dict ={}, density = None):
        self.componentA = components_dict['A']
        self.solvent1 = components_dict['solvent1']

        self.componentA_wtf = wtf_dict['A']
        self.solvent1_wtf = wtf_dict['solvent1']
        
        self.density = self.solvent1.density #Approximate density as density of solvent1 for now. 
                
    
    def wtf_to_volf(self):
        try:
            (self.componentA.density)
        except AttributeError:
            print(self.componentA.name, " has no density.")
        try:
            (self.solvent1.density)
        except AttributeError:
            print(self.solvent1.name, " has no density.")    
        self.componentA_volf = (self.componentA_wtf/self.componentA.density)/((self.componentA_wtf/self.componentA.density) + (self.solvent1_wtf/self.solvent1.density))
        self.solvent1_volf = 1.0 - self.componentA_volf

        # print("From A1stock: componentA_volf = ", self.componentA_volf)


    
    def wtf_to_molarity(self):
        try:
            (self.solvent1.density)
        except AttributeError:
            print(self.solvent1.name, " has no density.")   
        
        self.componentA_molarity = self.density*self.componentA_wtf*1000/self.componentA.mw
        # print("From A1stock: componentA_molarity = ", self.componentA_molarity)
        

    def real_init(self):
        pass

                

In [24]:
#Class: Components
#Purpose: Ease calling of chemical properties.


class Component:
    """Initialize your components to make calling of chemical properties easier"""
    def __init__(self,mw, density = 0.0, name = 'unnamed'):
#         Initialize properties of component. If density is unknown will default to 1 g/mL
        self.mw =float(mw)#component molecular weight (g/mol)
        if density != 0.0:
            self.density = float(density) #component density at 298.15 K, in g/mL
        else:
            density = 0.0
        self.name = str(name)

water = Component(18.01528,density = 0.997, name = 'water')
ethanol = Component(46.07, density = 0.79,name = 'ethanol')

pfh = Component(338.04, density = 1.67, name = 'pfh')
dlpc = Component(622.845, name = 'dlpc')
dmpc = Component(677.93, name = 'dmpc')
dppc = Component(734.04,name ='dppc')
dspc = Component(790.145, name ='dspc')
dspe_peg2000 = Component(2805.5, name = 'dspe_peg2000')
cholesterol = Component(386.654, name = 'cholesterol')
sodiumdodecylsulfate = Component(288.372, name = 'sodiumdodecylsulfate')
sodiumdodecylbenzenesulfonate = Component(342.4 , name = 'sodiumdodecylbenzenesulfonate')
dodecyltrimethylammoniumbromide = Component(308.34, name='dodecyltrimethylammoniumbromide')
five_cb = Component(249.356, name = 'five_cb')
hexadecane = Component(226.41 , density = .770, name = 'hexadecane')
oleic_acid = Component(282.47,density=0.895, name='oleic_acid')
hepes = Component(238.3012 ,name = 'hepes')
HAuCl4 = Component(339.79, name = 'HAuCl4')
MZ1 = Component(1268.5, name='MZ1')
MZ2 = Component(1190.7, name='MZ2')
Z1 = Component(1058.3, name='Z1')
Z2 = Component(980.3, name='Z2')
MAG3 = Component(1431.7, name='MAG3')
AG3 = Component(1221.6, name='AG3')

In [25]:
def dict_product(dicts):
    """
    Input a dictionary of key:component range array. where key is the role of the component in the sample, 
    e.g. 'stabilizer1':sodiumdodecylsulfate_range_list
    
    Generate a cartesian product in the form of a list of dictionaries containing all possible combinations. 
    >>> ]list(dict_product(dict(number=[1,2], character='ab')))
    [{'character': 'a', 'number': 1},
     {'character': 'a', 'number': 2},
     {'character': 'b', 'number': 1},
     {'character': 'b', 'number': 2}]
    """
    return (dict(zip(dicts, x)) for x in itertools.product(*dicts.values()))

def make_sample_list(sample_volume, components_dict, volf_dict_list, molarity_dict_list, stock_dict ):
    #All samples which are spanned by the initialization parameters. 
    raw_sample_object_list = []
    #Samples which can be made with this experimental setup
    safe_sample_object_list = []
    #Samples which cannot be made linked with associated reason tuple -> (sample_object, "string reason").  
    rejected_sample_object_tuple_list = []
    
    #Iterate through all possible volfs and molarities, and create samples with those target values
    #Add checks for whether volume of sample is too small too move, or if other issues arise.
    for volf_dict in volf_dict_list:
        for molarity_dict in molarity_dict_list:
            sample_ok = True
            new_sample = Oil_Surfacant_Mineralization(
                volume = sample_volume, components_dict = components_dict,volf_dict = volf_dict, 
                molarity_dict = molarity_dict, stock_dict = stock_dict)
            new_sample.transfer_ml_to_ul()
            for transfer in new_sample.converted_transfer_dict:
                if (new_sample.converted_transfer_dict[transfer] < 10.0) and new_sample.converted_transfer_dict[transfer] != .0:
#                     print("Rejected Sample:")
#                     print(new_sample.converted_transfer_dict[transfer], transfer)
                    rejected_sample_object_tuple_list.append(({'molarity_dict':molarity_dict,'volf_dict': volf_dict}, "Below pipetting volume", transfer))
                    sample_ok = False
                    break
#                     print("\n")

            if sample_ok:  
#                 print("Accepted Sample:")
                volume = 0.0
                for transfer in new_sample.assigned_volumes_dict:
                    volume+=new_sample.assigned_volumes_dict[transfer]
#                 print("Sample volume =", volume)
                safe_sample_object_list.append(new_sample)
#                 print("\n")
            else:
                pass

            
    return safe_sample_object_list, rejected_sample_object_tuple_list

def make_rejected_sample(sample_volume, components_dict, volf_dict, molarity_dict, stock_dict):
    sample_ok = True
    
    new_sample = Oil_Surfacant_Mineralization(
                volume = sample_volume, components_dict = components_dict,volf_dict = volf_dict, 
                molarity_dict = molarity_dict, stock_dict = stock_dict)
    new_sample.transfer_ml_to_ul()
    
    for transfer in new_sample.converted_transfer_dict:
                if (new_sample.converted_transfer_dict[transfer] < 10.0) and new_sample.converted_transfer_dict[transfer] != .0:
                    print("make_rejected_sample output: Rejected Sample,")
                    if transfer != 'solvent1':
                        print("volume = ",new_sample.converted_transfer_dict[transfer], "component =", transfer, 
                              "stock conc = ", stock_dict[transfer].componentA_wtf)
                    else:
                        print(new_sample.converted_transfer_dict[transfer], transfer, "solvent1")
                    print("\n")
                    sample_ok = False
                    break
      
    if sample_ok:
#         print("Accepted Sample:")
        volume = 0.0
        for transfer in new_sample.assigned_volumes_dict:
            volume+=new_sample.assigned_volumes_dict[transfer]
#         print("Sample volume =", volume)
#         print("\n")
    
    return new_sample, sample_ok



# End Loading Classes and Functions

In [33]:
sample_volume = 0.8 #Final volume of the sample that we are making in mL

#sample_stock_dict_list is a list of all samples to be made, each object is a dictionary:
#e.g. sample_stock_dict_list[n]= {'sample':sample object, 'stock_option':stock_dict}
sample_stock_dict_list= []

#Sample component definition
components_dict = {'stabilizer1':MZ1, 'solvent1':water, 
                   'hydrophobe':oleic_acid, 'precursor':HAuCl4, 'buffer': hepes}

#Stock instantiation variables
stabilizer1_stock = A1Stock(
    components_dict = {'A':MZ1, 'solvent1':water}, 
    wtf_dict = {'A':.0004,'solvent1':(1.0-.0004)})

hydrophobe_stock = A1Stock(
    components_dict = {'A':oleic_acid, 'solvent1':water}, 
    wtf_dict = {'A':3e-3, 'solvent1':(1.0-3e-3)})

precursor_stock = A1Stock(
    components_dict = {'A':HAuCl4, 'solvent1':water}, 
    wtf_dict ={'A':.001, 'solvent1':(1.0-.001)})

buffer_stock = A1Stock(
    components_dict = {'A':hepes, 'solvent1':water}, 
    wtf_dict ={'A':.001, 'solvent1':(1.0-.001)})

dilute_stabilizer1_stock = A1Stock(
    components_dict = {'A':MZ1, 'solvent1':water}, 
    wtf_dict = {'A':1e-5, 'solvent1':(1.0-1e-5)})

dilute_buffer_stock = A1Stock(
    components_dict = {'A':hepes, 'solvent1':water}, 
    wtf_dict = {'A':.0001, 'solvent1':(1.0-.0001)})

dilute_precursor_stock = A1Stock(
    components_dict = {'A':HAuCl4, 'solvent1':water},
    wtf_dict = {'A':.0001, 'solvent1':(1.0-0.0001)})

stock_dict  = {'stabilizer1':stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': buffer_stock}

dilute_stock_dict1 = {'stabilizer1':dilute_stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': buffer_stock}

dilute_stock_dict2 = {'stabilizer1':stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': dilute_buffer_stock}

dilute_stock_dict3 = {'stabilizer1':dilute_stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': dilute_buffer_stock}

dilute_stock_dict4 = {'stabilizer1':dilute_stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':dilute_precursor_stock, 'buffer': buffer_stock}

dilute_stock_dict5 = {'stabilizer1':stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':dilute_precursor_stock, 'buffer': dilute_buffer_stock}

dilute_stock_dict6 = {'stabilizer1':dilute_stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':dilute_precursor_stock, 'buffer': dilute_buffer_stock}

dilute_stock_dict7 = {'stabilizer1':stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':dilute_precursor_stock, 'buffer': buffer_stock}

#Experiment parameter definitions
# volf_dict_list = [{'hydrophobe':0, 'solvent1':float(1.0)}]
volf_dict_list = [{'hydrophobe':1e-3, 'solvent1':float(1.0-1e-3)}]


stabilizer1_molarity_list = np.logspace(-6.5, -4.5, num = 5)
precursor_molarity_list = np.logspace(-4.5, -3, 4)
buffer_molarity_list = np.logspace(-5,-3, num = 4)

#Append molarity = 0 control sample. 
stabilizer1_molarity_list = np.append(stabilizer1_molarity_list, [0.0]) 
# precursor_molarity_list = np.append(precursor_molarity_list, [0.0]) 
buffer_molarity_list = np.append(buffer_molarity_list, [0.0]) 


component_range_dict = {'stabilizer1':stabilizer1_molarity_list, 
                        'precursor':precursor_molarity_list, 'buffer':buffer_molarity_list}     


for component_key in components_dict:
    if ('hydrophobe' in component_key):
        stock_dict[component_key].wtf_to_volf()
        dilute_stock_dict1[component_key].wtf_to_volf()
        dilute_stock_dict2[component_key].wtf_to_volf()
        dilute_stock_dict3[component_key].wtf_to_volf()
        dilute_stock_dict4[component_key].wtf_to_volf()
        dilute_stock_dict5[component_key].wtf_to_volf()
        dilute_stock_dict6[component_key].wtf_to_volf()
        dilute_stock_dict7[component_key].wtf_to_volf()



    elif 'solvent1' not in component_key:
        stock_dict[component_key].wtf_to_molarity()
        dilute_stock_dict1[component_key].wtf_to_molarity()
        dilute_stock_dict2[component_key].wtf_to_molarity()
        dilute_stock_dict3[component_key].wtf_to_molarity()
        dilute_stock_dict4[component_key].wtf_to_molarity()
        dilute_stock_dict5[component_key].wtf_to_molarity()
        dilute_stock_dict6[component_key].wtf_to_molarity()
        dilute_stock_dict7[component_key].wtf_to_molarity()



component_molarity_range_dict_list = dict_product(component_range_dict)


# Make majority of samples with make_sample_list function:
sample_list1, rejected_sample_list = make_sample_list(sample_volume, components_dict, volf_dict_list, component_molarity_range_dict_list, stock_dict)


for sample in sample_list1:
    dict_of_tracked_properties = {'sample':sample}
    for stock_key in stock_dict:
        dict_of_tracked_properties[stock_key] = stock_dict[stock_key]
    
    sample_stock_dict_list.append(dict_of_tracked_properties)

print("Accepted samples = ",len(sample_list1),"Rejected samples = ", len(rejected_sample_list))
print("Trying to make rejected samples")

for rejected_sample in rejected_sample_list:
    stock_options = {'dilute_stock_dict1': dilute_stock_dict1,'dilute_stock_dict2': dilute_stock_dict2,
                     'dilute_stock_dict3': dilute_stock_dict3, 'dilute_stock_dict4':dilute_stock_dict4, 
                     'dilute_stock_dict5':dilute_stock_dict5, 'dilute_stock_dict6':dilute_stock_dict6,
                     'dilute_stock_dict7':dilute_stock_dict7}
    molarity_dict = rejected_sample[0]['molarity_dict']
    volf_dict = rejected_sample[0]['volf_dict']
    remake_boolean = False
    
    for stock_key in stock_options:
        stock_option = stock_options[stock_key]
        remade_sample, remake_boolean = make_rejected_sample(sample_volume, components_dict, volf_dict, molarity_dict, stock_option)
        if remake_boolean == True:
            print("Success! remade a rejected sample by using:", stock_key)
            print("_______________")
            break
    else:
        print("Still failed to make sample:", rejected_sample[0],'\n', remade_sample.converted_transfer_dict,'\n')
        print("_______________")

    dict_of_tracked_properties = {'sample':remade_sample}
    for stock_key in stock_option:
        dict_of_tracked_properties[stock_key] = stock_option[stock_key]
    sample_stock_dict_list.append(dict_of_tracked_properties)

Accepted samples =  27 Rejected samples =  93
Trying to make rejected samples
make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  0.8046838940669185 component = stabilizer1 stock conc =  0.0004


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  1.9121460381143431 component = buffer stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  0.8046838940669185 component = stabilizer1 stock conc =  0.0004


Success! remade a rejected sample by using: dilute_stock_dict6
_______________
make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  0.8046838940669185 component = stabilizer1 stock conc =  0.0004


make_re

Success! remade a rejected sample by using: dilute_stock_dict6
_______________
make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.046838940669186 component = stabilizer1 stock conc =  0.0004


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


Success! remade a rejected sample by using: dilute_stock_dict4
_______________
make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.046838940669186 component = stabilizer1 stock conc =  0.0004


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


Success! remade a rejected sample by using: dilute_stock_dict4
_______________
make_rejected_sample output: Rejected Sample,
vol

Success! remade a rejected sample by using: dilute_stock_dict5
_______________
make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.875395698748454 component = buffer stock conc =  0.001


Success! remade a rejected sample by using: dilute_stock_dict5
_______________
make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sample output: Rejected Sample,
volume =  8.621948454552566 component = precursor stock conc =  0.001


make_rejected_sampl

In [34]:
precursor_molarity_list
buffer_molarity_list
stabilizer1_molarity_list

stabilizer_precursor_ratios = []
buffer_precursor_ratios = []



for pre_conc in precursor_molarity_list:
    for stab_conc in stabilizer1_molarity_list:
        ratio = stab_conc/pre_conc
        stabilizer_precursor_ratios.append(ratio)
        
    for buff_conc in buffer_molarity_list:
        ratio = buff_conc/pre_conc
        buffer_precursor_ratios.append(ratio)
        
stabilizer_precursor_ratios.sort
buffer_precursor_ratios.sort
print(np.unique(np.asarray(stabilizer_precursor_ratios)))
print(np.unique(np.asarray(buffer_precursor_ratios)))  

[0.00000000e+00 3.16227766e-04 1.00000000e-03 3.16227766e-03
 1.00000000e-02 1.00000000e-02 3.16227766e-02 3.16227766e-02
 1.00000000e-01 1.00000000e-01 3.16227766e-01 1.00000000e+00]
[0.00000000e+00 1.00000000e-02 3.16227766e-02 4.64158883e-02
 1.00000000e-01 1.46779927e-01 2.15443469e-01 3.16227766e-01
 4.64158883e-01 6.81292069e-01 1.00000000e+00 1.46779927e+00
 2.15443469e+00 3.16227766e+00 6.81292069e+00 1.00000000e+01
 3.16227766e+01]


# Deck setup

In [42]:
robot.reset()
tiprack300_1 = labware.load('opentrons-tiprack-300ul','7')
tiprack300_2 = labware.load('opentrons-tiprack-300ul','8')
tiprack300_3 = labware.load('opentrons-tiprack-300ul','10')
tiprack300_4 = labware.load('opentrons-tiprack-300ul','11')

tipracks300 = [tiprack300_1, tiprack300_2]
tipracks50 = [tiprack300_3, tiprack300_4]

# tipracks_1000 = [labware.load('opentrons-tiprack-1000ul',slot) for slot in['7','10']]

# kwargs_300 = {'mount' : 'left', 'tip_racks' : tipracks300}
# kwargs_50 = {'mount' : 'right', 'tip_racks' : tipracks_50}
# kwargs_1000 = {'mount' : 'right', 'tip_racks' : tipracks_1000}
p300 = instruments.P300_Single(mount='left', tip_racks=tipracks300) 
p50 = instruments.P50_Single(mount='right', tip_racks=tipracks50)    
# p1000 = instruments.P1000_Single(**kwargs_1000)


medium_stock = labware.load('glass-20ml-vial',2)
large_stock = labware.load('agilent-201284-100',1)
sample_plate1 = labware.load('corning-48-3548',4)
sample_plate2 = labware.load('corning-48-3548',5)
sample_plate3 = labware.load('corning-48-3548',6)

sample_plate = sample_plate1.wells()+sample_plate2.wells()+sample_plate3.wells()


water_stock_position1 = large_stock.wells('A1')


stock_position_dict = {
    stabilizer1_stock:medium_stock.wells('A1'),
    dilute_stabilizer1_stock:medium_stock.wells('A2'), 
    buffer_stock:medium_stock.wells('A3'),
    dilute_buffer_stock:medium_stock.wells('A4'),
    precursor_stock:medium_stock.wells('B1'),
    dilute_precursor_stock:medium_stock.wells('B2'),
    hydrophobe_stock:medium_stock.wells('C1')
}

# Cell for checking how much of each stock is necessary to make.

In [43]:
solvent_sum_dict = {'solvent1':0, 'stabilizer1':{}, 'precursor':{}, 
                    'hydrophobe':{}, 'buffer':{}}


    
for sample_dict in sample_stock_dict_list:
    for component_key in sample_dict['sample'].converted_transfer_dict:
        if 'solvent' not in component_key:
#             print(component_key)
#             print(sample_dict[component_key].componentA_wtf)
#             print('\n')
            
            conc = sample_dict[component_key].componentA_wtf
            component_subdict = solvent_sum_dict[component_key]
            if conc not in component_subdict:
                component_subdict[conc] = sample_dict['sample'].converted_transfer_dict[component_key]/1000
            else:
#                 if component_key == 'hydrophobe':
#                     print(sample_dict['sample'].converted_transfer_dict[component_key]/1000)
                component_subdict[conc] += sample_dict['sample'].converted_transfer_dict[component_key]/1000
            
        elif 'solvent' in component_key:
            solvent_sum_dict[component_key] += sample_dict['sample'].converted_transfer_dict[component_key]/1000
    
        
    
print(solvent_sum_dict, '\n')


{'solvent1': 61.4934509506679, 'stabilizer1': {0.0004: 2.1182945684748598, 1e-05: 9.116925389152975}, 'precursor': {0.001: 11.584032881400866, 0.0001: 2.586584536365771}, 'hydrophobe': {0.003: 28.735999999999933}, 'buffer': {0.001: 5.4954611179969435, 0.0001: 3.412928768231185}} 



In [44]:


for stock in stock_position_dict:
    print(stock.componentA.name)
    print("weight fraction = ", stock.componentA_wtf)
    print((stock_position_dict[stock]))
    
    print(stock_position_dict[stock].parent.wells)

    if hasattr(stock, 'componentA_molarity'):
        print("molarity = ", stock.componentA_molarity)
    elif hasattr(stock, 'componentA_volf'):
        print("volume fraction = ", stock.componentA_volf)

    print("____________________________________________________________")

MZ1
weight fraction =  0.0004
<Well A1>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.00031438707134410723
____________________________________________________________
MZ1
weight fraction =  1e-05
<Well A2>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  7.859676783602681e-06
____________________________________________________________
hepes
weight fraction =  0.001
<Well A3>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.004183780862202961
____________________________________________________________
hepes
weight fraction =  0.0001
<Well A4>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.00041837808622029605
____________________________________________________________
HAuCl4
weight fraction =  0.001
<Well B1>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.002934165219694517
__

# Export sample information

In [45]:
# for sample_dict in enumerate(sample_stock_dict_list):
#     print(sample_dict[1]['sample'].molarity_dict)
#     print(sample_plate)
# #     print(sample_plate.well(sample_dict[0]))
#     print('\n')
    
sample_info_filename = '2020_07_30_OuzoMZ1.csv'

with open(sample_info_filename, 'w',newline = '') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=' ')
    
    sample = sample_stock_dict_list[0]['sample']
    header1 = [component.name for component in sample.components_dict.values()]
    location_keys = ["Slot","LabwareType","WellCode", "WellIndex"]
    location_dict = dict.fromkeys(["Slot","LabwareType","WellCode", "WellIndex"], None)
    UID_dict = {'UID':None}
    
    header1.extend(key for key in location_dict.keys())
    header1.extend(UID_dict.keys())
    
    header2 = [key for key in sample.components_dict.keys()]
    header2.extend(key for key in location_dict.keys())
    header2.extend(UID_dict.keys())

    
    csvwriter.writerow(header1)
    csvwriter.writerow(header2)
    csvwriter = csv.DictWriter(csvfile,delimiter = ' ', fieldnames = header2)
    
    for sample_dict in sample_stock_dict_list:
        dict_to_header = None
        sample = sample_dict['sample']
        sample_well_index = sample_stock_dict_list.index(sample_dict)
        s = (sample_plate.well(sample_well_index))
        location = s.get_path()
        location.append(sample_well_index)
        dict_to_header = sample.molarity_dict
        

        for i, key in enumerate(location_keys):
            dict_to_header[key] = location[i]
        well_code = dict_to_header["WellCode"]
        slot = dict_to_header["Slot"]
        UID = str(datetime.datetime.now(pacific).date())+"_Emuls"+components_dict['stabilizer1'].name+"_"+"S"+slot+"_"+well_code
        UID_dict["UID"] = UID
        
#         dict_to_header.update({'Well':location})
        dict_to_header.update(sample.volf_dict)
        dict_to_header.update(UID_dict)
        csvwriter.writerow(dict_to_header)
       

In [46]:
# # print(sample_stock_dict_list[0])
# for sample in sample_stock_dict_list:
#     for key in sample:
#         print(key, sample_stock_dict_list[0][key], '\n')
# #         if(sample_stock_dict_list[0][key] in stock_position_dict):
            
# robot.head_speed(300)
# robot.home()
# robot.reset()

    
def pipette_stock_to_well(component_key, stock_position_dict, sample_stock_dict_list, destination_offset = 0, mix = False,
                         sample_index_offset = 0):
    pipette = p50
    if pipette.tip_attached:
        pipette.drop_tip()
    pipette = p300
    if pipette.tip_attached:
        pipette.drop_tip()
    remembered_source = None
    for sample_dict in sample_stock_dict_list[sample_index_offset:]:
        if 'solvent' in component_key:
            source = water_stock_position1
            volume = sample_dict['sample'].converted_transfer_dict[component_key]
            destination = sample_plate[sample_stock_dict_list.index(sample_dict)+destination_offset]
        else:
            stock_used = sample_dict[component_key]
            source = stock_position_dict[stock_used]
            volume = sample_dict['sample'].converted_transfer_dict[component_key]
            destination = sample_plate[sample_stock_dict_list.index(sample_dict)+destination_offset]
        print("Transferring ", volume, "from ", source, "to ", destination, " ", datetime.datetime.now(pacific))


        if volume == 0:
            # if no volume is to be pipetted then do nothing, go to next transfer step
            next
        else:
            if volume > p300.min_volume:
                if (pipette == p50) and (pipette.tip_attached):
                    pipette.drop_tip()
                else:
                    pass
                pipette = p300
            else:
                if (pipette == p300) and (pipette.tip_attached):
                    pipette.drop_tip()
                else:
                    pass
                pipette = p50


            if (source != remembered_source):
                pipette.drop_tip()

            if not pipette.tip_attached:
                pipette.pick_up_tip()

            if sample_stock_dict_list.index(sample_dict) == 0:
                pipette.mix(3, pipette.max_volume, source)
            
#             if pipette == p50:
#                 rate = 1
#             elif pipette == p300:
#                 rate = 1.5
            
            pipette.transfer(volume, source, destination.top(-3), new_tip ='never', rate = 1)
            pipette.blow_out()

            if mix == True:
                pipette.mix(3, 50, destination, rate = 3.0)
                if not pipette.tip_attached: print("pipette is dropped after mix")
                pipette.drop_tip()

            remembered_source = source


    return

In [47]:
pipette_stock_to_well('stabilizer1', stock_position_dict, sample_stock_dict_list)

Cannot drop tip without a tip attached.


Transferring  25.44633901705116 from  <Well A1> to  <Well A1>   2020-07-30 02:05:36.357201-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well B1>   2020-07-30 02:06:10.899696-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well C1>   2020-07-30 02:06:18.367297-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well D1>   2020-07-30 02:06:25.796420-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well E1>   2020-07-30 02:06:33.208558-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well F1>   2020-07-30 02:06:40.611307-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well A2>   2020-07-30 02:06:47.995085-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well B2>   2020-07-30 02:06:55.405039-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well C2>   2020-07-30 02:07:02.830728-07:00
Transferring  80.46838940669186 from  <Well A1> to  <Well D2>   2020-07-30 02:07:10.206421-07:00
Transferring  80.4683894066918

Transferring  321.87355762676737 from  <Well A2> to  <Well C7>   2020-07-30 02:17:29.622406-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well D7>   2020-07-30 02:17:43.614465-07:00


Cannot drop tip without a tip attached.


Transferring  25.44633901705116 from  <Well A1> to  <Well E7>   2020-07-30 02:18:17.966371-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well F7>   2020-07-30 02:18:25.091407-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well A8>   2020-07-30 02:18:32.179675-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well B8>   2020-07-30 02:18:39.416980-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well C8>   2020-07-30 02:18:46.764467-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well D8>   2020-07-30 02:18:54.055378-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well E8>   2020-07-30 02:19:01.272449-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well F8>   2020-07-30 02:19:08.464531-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well A1>   2020-07-30 02:19:15.631696-07:00
Transferring  25.44633901705116 from  <Well A1> to  <Well B1>   2020-07-30 02:19:22.965771-07:00
Transferring  80.4683894066918

In [48]:
pipette_stock_to_well('hydrophobe', stock_position_dict, sample_stock_dict_list)

Cannot drop tip without a tip attached.


Transferring  239.46666666666667 from  <Well C1> to  <Well A1>   2020-07-30 02:46:22.060722-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well B1>   2020-07-30 02:46:55.348880-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well C1>   2020-07-30 02:47:04.049544-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well D1>   2020-07-30 02:47:12.687518-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well E1>   2020-07-30 02:47:21.265270-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well F1>   2020-07-30 02:47:29.787168-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well A2>   2020-07-30 02:47:38.256014-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well B2>   2020-07-30 02:47:46.828568-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well C2>   2020-07-30 02:47:55.498540-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well D2>   2020-07-30 02:48:04.110062-07:00
Transferring  239.46

Transferring  239.46666666666667 from  <Well C1> to  <Well A7>   2020-07-30 02:58:37.865056-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well B7>   2020-07-30 02:58:46.317682-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well C7>   2020-07-30 02:58:54.918573-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well D7>   2020-07-30 02:59:03.450657-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well E7>   2020-07-30 02:59:11.914086-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well F7>   2020-07-30 02:59:20.312840-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well A8>   2020-07-30 02:59:28.643899-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well B8>   2020-07-30 02:59:37.115218-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well C8>   2020-07-30 02:59:45.724758-07:00
Transferring  239.46666666666667 from  <Well C1> to  <Well D8>   2020-07-30 02:59:54.271308-07:00
Transferring  239.46

In [49]:
pipette_stock_to_well('buffer', stock_position_dict, sample_stock_dict_list)

Cannot drop tip without a tip attached.


Transferring  41.195937569205654 from  <Well A3> to  <Well A1>   2020-07-30 03:03:56.277426-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well B1>   2020-07-30 03:04:27.532451-07:00
Transferring  0.0 from  <Well A3> to  <Well C1>   2020-07-30 03:04:35.770427-07:00
Transferring  41.195937569205654 from  <Well A3> to  <Well D1>   2020-07-30 03:04:35.772928-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well E1>   2020-07-30 03:04:42.495701-07:00
Transferring  0.0 from  <Well A3> to  <Well F1>   2020-07-30 03:04:50.686889-07:00
Transferring  41.195937569205654 from  <Well A3> to  <Well A2>   2020-07-30 03:04:50.689396-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well B2>   2020-07-30 03:04:57.378410-07:00
Transferring  0.0 from  <Well A3> to  <Well C2>   2020-07-30 03:05:05.557403-07:00
Transferring  41.195937569205654 from  <Well A3> to  <Well D2>   2020-07-30 03:05:05.560021-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well E2>   

Cannot drop tip without a tip attached.


Transferring  88.75395698748454 from  <Well A4> to  <Well E5>   2020-07-30 03:07:07.874506-07:00
Transferring  41.195937569205654 from  <Well A3> to  <Well F5>   2020-07-30 03:07:34.169163-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well A6>   2020-07-30 03:07:57.043020-07:00
Transferring  0.0 from  <Well A3> to  <Well B6>   2020-07-30 03:08:04.983698-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well C6>   2020-07-30 03:08:04.986242-07:00


Cannot drop tip without a tip attached.


...

Transferring  88.75395698748454 from  <Well A4> to  <Well D8>   2020-07-30 03:26:51.579313-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well E8>   2020-07-30 03:27:17.269552-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well F8>   2020-07-30 03:27:43.643734-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well A1>   2020-07-30 03:28:09.341735-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well B1>   2020-07-30 03:28:35.909157-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well C1>   2020-07-30 03:29:01.584852-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well D1>   2020-07-30 03:29:27.938269-07:00
Transferring  411.9593756920566 from  <Well A4> to  <Well E1>   2020-07-30 03:29:53.610835-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well F1>   2020-07-30 03:30:08.167691-07:00
Transferring  0.0 from  <Well A4> to  <Well A2>   2020-07-30 03:30:32.030066-07:00
Transferring  19.121460381143432 from  <

Cannot drop tip without a tip attached.


Transferring  88.75395698748454 from  <Well A4> to  <Well C2>   2020-07-30 03:31:06.122135-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well D2>   2020-07-30 03:31:31.773013-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well E2>   2020-07-30 03:31:58.067880-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well F2>   2020-07-30 03:32:23.695917-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well A3>   2020-07-30 03:32:49.972846-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well B3>   2020-07-30 03:33:15.834548-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well C3>   2020-07-30 03:33:42.097134-07:00
Transferring  41.195937569205654 from  <Well A3> to  <Well D3>   2020-07-30 03:34:07.737212-07:00
Transferring  191.21460381143427 from  <Well A3> to  <Well E3>   2020-07-30 03:34:30.126914-07:00
Transferring  0.0 from  <Well A3> to  <Well F3>   2020-07-30 03:34:37.865998-07:00
Transferring  19.121460381143432 from  

Cannot drop tip without a tip attached.


Transferring  88.75395698748454 from  <Well A4> to  <Well B4>   2020-07-30 03:35:11.940326-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well C4>   2020-07-30 03:35:37.620464-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well D4>   2020-07-30 03:36:03.970401-07:00
Transferring  19.121460381143432 from  <Well A4> to  <Well E4>   2020-07-30 03:36:29.630374-07:00
Transferring  88.75395698748454 from  <Well A4> to  <Well F4>   2020-07-30 03:36:55.989431-07:00


In [50]:
pipette_stock_to_well('solvent1', stock_position_dict, sample_stock_dict_list)

Cannot drop tip without a tip attached.


Transferring  704.4900468645425 from  <Well A1> to  <Well A1>   2020-07-30 03:37:31.264083-07:00
Transferring  554.471380622314 from  <Well A1> to  <Well B1>   2020-07-30 03:38:19.156954-07:00
Transferring  745.6859844337482 from  <Well A1> to  <Well C1>   2020-07-30 03:38:35.798915-07:00
Transferring  645.5355573039717 from  <Well A1> to  <Well D1>   2020-07-30 03:38:59.276658-07:00
Transferring  495.51689106174314 from  <Well A1> to  <Well E1>   2020-07-30 03:39:21.579443-07:00
Transferring  686.7314948731774 from  <Well A1> to  <Well F1>   2020-07-30 03:39:37.242897-07:00
Transferring  459.10509199994874 from  <Well A1> to  <Well A2>   2020-07-30 03:39:59.513172-07:00
Transferring  309.0864257577202 from  <Well A1> to  <Well B2>   2020-07-30 03:40:15.147667-07:00
Transferring  500.30102956915437 from  <Well A1> to  <Well C2>   2020-07-30 03:40:29.385197-07:00
Transferring  649.4679964749018 from  <Well A1> to  <Well D2>   2020-07-30 03:40:45.371720-07:00
Transferring  499.4493302326

Transferring  12.659207148003837 from  <Well A1> to  <Well B7>   2020-07-30 04:02:22.784921-07:00
Transferring  203.87381095943812 from  <Well A1> to  <Well C7>   2020-07-30 04:02:49.002654-07:00
Transferring  667.610034492034 from  <Well A1> to  <Well D7>   2020-07-30 04:03:16.051711-07:00
Transferring  597.9775378856929 from  <Well A1> to  <Well E7>   2020-07-30 04:03:40.348548-07:00
Transferring  274.7721191811208 from  <Well A1> to  <Well F7>   2020-07-30 04:03:58.300362-07:00
Transferring  495.51689106174314 from  <Well A1> to  <Well A8>   2020-07-30 04:04:07.442975-07:00
Transferring  686.7314948731774 from  <Well A1> to  <Well B8>   2020-07-30 04:04:24.574365-07:00
Transferring  726.5645240526047 from  <Well A1> to  <Well C8>   2020-07-30 04:04:49.382853-07:00
Transferring  656.9320274462636 from  <Well A1> to  <Well D8>   2020-07-30 04:05:14.502732-07:00
Transferring  667.610034492034 from  <Well A1> to  <Well E8>   2020-07-30 04:05:38.880206-07:00
Transferring  597.97753788569

In [51]:
pipette_stock_to_well('precursor', stock_position_dict, sample_stock_dict_list, mix=True, sample_index_offset=0, destination_offset=0)

Cannot drop tip without a tip attached.


Transferring  27.264994984954868 from  <Well B1> to  <Well A1>   2020-07-30 04:22:38.516057-07:00
Transferring  27.264994984954868 from  <Well B1> to  <Well B1>   2020-07-30 04:23:29.545442-07:00
Transferring  27.264994984954868 from  <Well B1> to  <Well C1>   2020-07-30 04:23:59.385752-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well D1>   2020-07-30 04:24:29.216308-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well E1>   2020-07-30 04:24:58.380923-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well F1>   2020-07-30 04:25:24.633565-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well A2>   2020-07-30 04:25:50.955678-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well B2>   2020-07-30 04:26:19.080236-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well C2>   2020-07-30 04:26:47.173097-07:00
Transferring  27.264994984954868 from  <Well B1> to  <Well D2>   2020-07-30 04:27:15.234900-07:00
Transferring  27.264994

Cannot drop tip without a tip attached.


Transferring  86.21948454552567 from  <Well B2> to  <Well D5>   2020-07-30 04:35:50.345232-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well E5>   2020-07-30 04:36:23.906086-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well F5>   2020-07-30 04:36:50.031985-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well A6>   2020-07-30 04:37:16.165402-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well B6>   2020-07-30 04:37:42.245844-07:00


Cannot drop tip without a tip attached.


...

Transferring  27.264994984954868 from  <Well B1> to  <Well B2>   2020-07-30 05:11:36.075709-07:00
Transferring  27.264994984954868 from  <Well B1> to  <Well C2>   2020-07-30 05:12:16.136770-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well D2>   2020-07-30 05:12:45.519295-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well E2>   2020-07-30 05:13:14.147067-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well F2>   2020-07-30 05:13:39.820855-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well A3>   2020-07-30 05:14:07.402527-07:00


Cannot drop tip without a tip attached.


Transferring  86.21948454552567 from  <Well B2> to  <Well B3>   2020-07-30 05:14:34.910012-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well C3>   2020-07-30 05:15:07.995060-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well D3>   2020-07-30 05:15:33.627824-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well E3>   2020-07-30 05:15:59.261951-07:00
Transferring  86.21948454552567 from  <Well B2> to  <Well F3>   2020-07-30 05:16:24.922775-07:00


Cannot drop tip without a tip attached.


Transferring  27.264994984954868 from  <Well B1> to  <Well A4>   2020-07-30 05:16:50.617598-07:00
Transferring  27.264994984954868 from  <Well B1> to  <Well B4>   2020-07-30 05:17:30.697383-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well C4>   2020-07-30 05:18:00.180396-07:00
Transferring  86.21948454552566 from  <Well B1> to  <Well D4>   2020-07-30 05:18:28.847329-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well E4>   2020-07-30 05:18:54.570353-07:00
Transferring  272.64994984954865 from  <Well B1> to  <Well F4>   2020-07-30 05:19:22.179898-07:00
